# Random Forest

**Random Forest** или Случайный лес - модель, используемая как для регрессии, так и для классификации. Алгоритм основан на другом алгоритме - Decision Tree или Дерево решений, т.е. деревья используются для создания леса. Термин **"случайный"** говорит о том, что каждое дерево построено на основе случайной подвыборки данных.

Для начала разберемся в том, как работает алгоритм Decision Tree.

 ## Подварительная обработка данных

In [173]:
import pandas as pd
from matplotlib import pyplot as plt

In [174]:
df_train = pd.read_csv('data/train_data.csv', header=None, index_col=False)
df_test = pd.read_csv('data/test_data.csv', header=None, index_col=False)

In [175]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       32561 non-null  int64 
 1   1       32561 non-null  object
 2   2       32561 non-null  int64 
 3   3       32561 non-null  object
 4   4       32561 non-null  int64 
 5   5       32561 non-null  object
 6   6       32561 non-null  object
 7   7       32561 non-null  object
 8   8       32561 non-null  object
 9   9       32561 non-null  object
 10  10      32561 non-null  int64 
 11  11      32561 non-null  int64 
 12  12      32561 non-null  int64 
 13  13      32561 non-null  object
 14  14      32561 non-null  int64 
dtypes: int64(7), object(8)
memory usage: 3.7+ MB


In [176]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16281 entries, 0 to 16280
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       16281 non-null  int64 
 1   1       16281 non-null  object
 2   2       16281 non-null  int64 
 3   3       16281 non-null  object
 4   4       16281 non-null  int64 
 5   5       16281 non-null  object
 6   6       16281 non-null  object
 7   7       16281 non-null  object
 8   8       16281 non-null  object
 9   9       16281 non-null  object
 10  10      16281 non-null  int64 
 11  11      16281 non-null  int64 
 12  12      16281 non-null  int64 
 13  13      16281 non-null  object
dtypes: int64(6), object(8)
memory usage: 1.7+ MB


In [177]:
df_train.head(8)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,0
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,0
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,1


In [167]:
{i:df_train.iloc[:, i].unique().size for i in range(df_train.shape[1])}

{0: 73,
 1: 9,
 2: 21648,
 3: 16,
 4: 16,
 5: 7,
 6: 15,
 7: 6,
 8: 5,
 9: 2,
 10: 119,
 11: 92,
 12: 94,
 13: 42,
 14: 2}

In [187]:
from sklearn import preprocessing

In [192]:
df_train_encoded = df_train.apply(preprocessing.LabelEncoder().fit_transform)

In [193]:
df_train_encoded

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,22,7,2671,9,12,4,1,1,4,1,25,0,39,39,0
1,33,6,2926,9,12,2,4,0,4,1,0,0,12,39,0
2,21,4,14086,11,8,0,6,1,4,1,0,0,39,39,0
3,36,4,15336,1,6,2,6,0,2,1,0,0,39,39,0
4,11,4,19355,9,12,2,10,5,2,0,0,0,39,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,10,4,16528,7,11,2,13,5,4,0,0,0,37,39,0
32557,23,4,8080,11,8,2,7,0,4,1,0,0,39,39,1
32558,41,4,7883,11,8,6,1,4,4,0,0,0,39,39,0
32559,5,4,12881,11,8,4,1,3,4,1,0,0,19,39,0


## Decision Tree

Деревья решений состоят из двух элементов: узлы и ветви. Сам алгоритм основан на рекурсивном разбиении выборки по признаку, который даёт наибольший прирост информации (Gain), т.е. наилучшим образом разделяет выборку на данный момент. Разбиение будет происходить до тех пор, пока мы не дойдём до листьев - узлов, содержащих только экземпляры одного класса (pure, чистых). Пример дерева решений изображен на рисунке ниже.

<img src="images/decision_tree.svg" width=70%/>

Как видно из рисунка, существует несколько типов узлов:

1. **Корневой узел** &mdash; узел на вершине дерева. Содержит признак, который лучше всего делит данные (единственный признак, который классифицирует целевую переменную наиболее точно). Имеет 2 дочерних узла.
2. **Узел** &mdash; внутренний узел дерева, узел проверки. Каждый такой узел имеет 2 дочерних узла.
3. **Лист** &mdash; конечный узел дерева, узел решения. Не имеет потомков.

Для оценки качества разбиения выборки по признаку используется **энтропия Шеннона**. Энтропия рассматривается как мера неоднородности подмножества по представленным в нем классам. Если классы представлены в равных долях, неопределенность классификации наибольшая, то и энтропия тоже максимальная. Логарифм от единицы будет обращать энтропию в ноль, если все примеры узла относятся к одному классу. Может быть расчитана по следующей формуле:
$$H(T) = -\sum_{i=1}^n p_i log_2 p_i = -\sum_{i=1}^n \frac{N_i}{N} log_2 \left(\frac{N_i}{N}\right),$$
где $T$ - вектор значений целевой переменной (target), $n$ — число классов в исходном подмножестве, $N_i$ — число экземпляров i-го класса в подмножестве, $N$ — общее число экземпляров в подмножестве.

In [71]:
import numpy as np
from collections import Counter

In [72]:
def entropy(t):
    counts = np.bincount(t)
    probability = counts / len(t)
    
    entropy = 0
    for prob in probability:
        if prob > 0:
            entropy += prob * np.log2(prob)
    return -entropy

In [73]:
s = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
print(f'Entropy: {np.round(entropy(s), 5)}')

Entropy: 0.88129


На практике, однако, говорят не об энтропии, а о величине, обратной ей, которая называется **Приростом информации (Inforamtion Gain, IG)**. Тогда лучшим атрибутом разбиения будет тот, который обеспечит максимальный прирост информации результирующего узла относительно исходного. 
Прирост информации представляет собой разницу между энтропией исходной выборки и взвешенным средним суммы всех значений энтропии для разделённых подмножеств. Чем выше значение информационного прироста, тем лучше принятое решение. Может быть расчитан по следующей формуле:
$$IG(T, A) = H(T) - H(T|A)$$
$$IG(T, A) = H(T) - \sum_{v \in Values(A)}\frac{|T_{v}|}{|T|}H(T_{v}),$$
где $T$ - вектор значений целевой переменной (target), $A$ - признак, по которому было совершено разбиение, $v$ - каждое уникальное значение признака $A$.

Если объяснить немного по-другому, мы находим энтропию каждого набора после разбиения, взвешиваем ее по количеству элементов в каждом разбиении, затем вычитаем из текущей энтропии. Если результат положительный, значит, мы снизили энтропию при разбиении. Чем выше результат, тем больше мы понизили энтропию.

In [74]:
def information_gain(parent, left, right):
    num_left = len(left) / len(parent)
    num_right = len(right) / len(parent)
    
    gain = entropy(parent) - (num_left * entropy(left) + num_right * entropy(right))
    return gain

In [75]:
parent = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
left_child = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
right_child = [0, 0, 0, 0, 1, 1, 1, 1]
print(f'Information gain: {np.round(information_gain(parent, left_child, right_child), 5)}')

Information gain: 0.18094


## Реализация Decision Tree на Python
Для реализации необходимы 2 класса:
* Node - реализует один узел дерева решений
* DecisionTree - реализует алгоритм дерева решений

### Класс, реализующий один узел дерева
Класс, необходимый для хранения данных о признаке(feature) и пороговом значении(threshold), по которым производилось разбиение выборки, поддеревья левое(data_left) и правое(data_right), информационном приросте(gain) и значении классификации(value), которое устанавливается только для листовых узлов.

In [76]:
class Node:
    def __init__(self, feature=None, threshold=None, data_left=None, data_right=None, gain=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.data_left = data_left
        self.data_right = data_right
        self.gain = gain
        self.value = value

### Класс, реализующий дерево решений
Класс содержит следующие методы:
1. Конструктор **\_\_init__**, в котором задаются значения для min_samples_split и max_depth. Это гиперпараметры. Первый используется для указания минимального количества образцов, необходимых для разбиения узла, а второй задает максимальную глубину дерева. Оба параметра используются в рекурсивных функциях в качестве условий выхода.
1. Метод **\_entropy(t)** вычисляет энтропию входного вектора t
1. Метод **\_information_gain(parent, left, right)** вычисляет значение информационного прироста при разделении родительского и двух дочерних векторов.
1. Метод **\_best_split(X, y)** вычисляет наилучшие параметры разбиения для входных признаков X и целевой переменной y. Это делается путем итерации по каждому столбцу в X и по каждому пороговому значению в каждом столбце, чтобы найти оптимальное разбиение с использованием информационного прироста.
1. Метод **\_build(X, y, depth)** рекурсивно строит дерево решений до достижения критериев останова (гиперпараметры в конструкторе).
1. Метод **fit(X, y)** вызывает функцию **\_build()** и сохраняет построенное дерево
1. Метод **\_predict(x)** обходит дерево для классификации одного экземпляра
1. Метод **predict(X)** применяет функцию **\_predict()** к каждому экземпляру в матрице X

In [98]:
class DecisionTree:
    '''
    Класс, реализующий классификатор на основе алгоритма дерева решений
    '''
    def __init__(self, min_samples_split=2, max_depth=5):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.max_features = None
        self.root = None
    
    @staticmethod
    def _entropy(t):
        '''
        Вспомогательная функция, вычисляет энтропию массива целочисленных значений
        '''
        counts = np.bincount(np.array(t, dtype=np.int))
        probability = counts / len(t)
    
        entropy = 0
        for prob in probability:
            if prob > 0:
                entropy += prob * np.log2(prob)
        return -entropy
    
    def _information_gain(self, parent, left, right):
        '''
        Вспомогательная функция, вычисляет информационный прирост от родителя и двух узлов-потомков
        '''
        num_left = len(left) / len(parent)
        num_right = len(right) / len(parent)
    
        gain = self._entropy(parent) - (num_left * self._entropy(left) + num_right * self._entropy(right))
        return gain
    
    def _best_split(self, X, y):
        '''
        Вспомогательная функция, вычисляет лучшее разбиение для заданных признаков и меток
        '''
        best_split = {}
        best_info_gain = -1
        # Random Subspace - выбор max_features признаков без возврата из исходной выборки
        new_features = np.random.choice(X.shape[1], self.max_features, replace=False)
        
        # for every feature
        for f_idx in new_features:
            X_curr = X[:, f_idx]
            # for every unuque value of feature
            for threshold in np.unique(X_curr):
                df = np.concatenate((X, y.reshape(1, -1).T), axis=1)
                df_left = np.array([row for row in df if row[f_idx] <= threshold])
                df_right = np.array([row for row in df if row[f_idx] > threshold])
            
                if len(df_left) > 0 and len(df_right) > 0:
                    y = df[:, -1]
                    y_left = df_left[:, -1]
                    y_right = df_right[:, -1]
                
                    gain = self._information_gain(y, y_left, y_right)
                    if gain > best_info_gain:
                        best_split = {
                            'feature_index': f_idx,
                            'threshold': threshold,
                            'df_left': df_left,
                            'df_right': df_right,
                            'gain': gain
                        }
                        best_info_gain = gain
        return best_split
    
    def _build(self, X, y, depth=0):
        '''
        Вспомогательная рекурсивная функция, используется для построения дерева решений по входным данным
        '''
        n_rows, n_cols = X.shape
        
        if n_rows >= self.min_samples_split and depth <= self.max_depth:
            best = self._best_split(X, y)
            if best['gain'] > 0: # impure split
                left = self._build(
                    X=best['df_left'][:, :-1], 
                    y=best['df_left'][:, -1], 
                    depth=depth + 1
                )
                right = self._build(
                    X=best['df_right'][:, :-1], 
                    y=best['df_right'][:, -1], 
                    depth=depth + 1
                )
                return Node(
                    feature=best['feature_index'], 
                    threshold=best['threshold'], 
                    data_left=left, 
                    data_right=right, 
                    gain=best['gain']
                )
        # лист (pure), value - самое часто встречающееся значение в y
        return Node(
            value=int(Counter(y).most_common(1)[0][0])
        )
    
    def fit(self, X, y, max_features=-1):
        '''
        Функция для обучения классификатора дерева решений
        '''
        if max_features == -1: # не использовать Random Subspace при каждом разбиении узла
            self.max_features = X.shape[1]
        elif max_features == 0: # использовать рекомендуемое число признаков для Random Subspace (sqrt(num_of_features))
            self.max_features = int(np.round(np.sqrt(X.shape[1])))
        else: # использовать заданное число признаков для Random Subspace
            self.max_features = int(max_features)
#         print(self.max_features)
        self.root = self._build(X, y)
    
    def _predict(self, x, tree):
        '''
        Вспомогательная рекурсивная функция для предсказания одного значения (обход дерева)
        '''
        # leaf
        if tree.value != None:
            return tree.value
        feature_value = x[tree.feature]
        
        # go left
        if feature_value <= tree.threshold:
            return self._predict(x=x, tree=tree.data_left)
        
        # go right
        if feature_value > tree.threshold:
            return self._predict(x=x, tree=tree.data_right)
        
    def predict(self, X):
        '''
        Функция для предсказания значений меток
        '''
        return np.array([self._predict(x, self.root) for x in X])

## Тестирование

In [99]:
from sklearn.datasets import load_iris

iris = load_iris()

X = iris['data']
y = iris['target']

In [100]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [101]:
model = DecisionTree(min_samples_split=2, max_depth=5)

In [102]:
model.fit(X_train, y_train, max_features=-1)

In [103]:
y_pred = model.predict(X_test)

In [104]:
y_pred

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0])

In [105]:
y_test

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0])

In [106]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

1.0

Сравним построенную модель с классификатором из библиотеки Scikit-Learn DecisionTreeClassifier.

In [107]:
from sklearn.tree import DecisionTreeClassifier

sk_model = DecisionTreeClassifier()
sk_model.fit(X_train, y_train)
sk_y_pred = sk_model.predict(X_test)

accuracy_score(y_test, sk_y_pred)

1.0

### Тестирование на настоящих данных

In [203]:
df_train_encoded.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,22,7,2671,9,12,4,1,1,4,1,25,0,39,39,0
1,33,6,2926,9,12,2,4,0,4,1,0,0,12,39,0
2,21,4,14086,11,8,0,6,1,4,1,0,0,39,39,0
3,36,4,15336,1,6,2,6,0,2,1,0,0,39,39,0
4,11,4,19355,9,12,2,10,5,2,0,0,0,39,5,0


In [204]:
X_tr_enc = df_train_encoded.iloc[:, :-1]
y_tr_enc = df_train_encoded.iloc[:, -1]

In [213]:
X_train_enc, X_test_enc, y_train_enc, y_test_enc = train_test_split(
    X_tr_enc.values,
    y_tr_enc.values,
    test_size=0.2,
    random_state=42)

In [214]:
model_enc = DecisionTree(min_samples_split=2, max_depth=5)

In [215]:
model_enc.fit(X_train_enc, y_train_enc, max_features=-1)

KeyboardInterrupt: 

## Random Forest

Алгоритм Случайного леса состоит из трёх компонентов:
$$Random Forest = Decision Tree + Bagging + Random Subspace$$

<img src="images/comparison.png" width=70%/>

## Реализация Random Forest на Python

Класс содержит следующие методы:

1. **\_\_init__()** - конструктор, хранит значения гиперпараметров для количества деревьев в лесе, минимального количества элементовв разбиении дерева и максимальной глубины. В нём также будут храниться индивидуально обученные деревья решений после обучения модели.
1. **\_sample(X, y)** применяет bootstrap-выборку к входным признакам и входной целевой переменной
1. **fit(X, y)** метод для обучения модели классификатора
1. **predict(X)** делает прогнозы с помощью отдельных деревьев решений, а затем применяет мажоритарное голосование для окончательного прогноза

In [112]:
class RandomForest:
    '''
    Класс, реализующий алгоритм Слечайного леса
    '''
    def __init__(self, num_trees=25, min_samples_split=2, max_depth=5):
        self.num_trees = num_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.max_features = None
        # Храним все построенные деревья
        self.decision_trees = []

    @staticmethod
    def _sample(X, y):
        '''
        Вспомогательная функция для создания bootstrap-выборки
        '''
        n_rows, n_cols = X.shape
        # Sample with replacement
        samples = np.random.choice(a=n_rows, size=n_rows, replace=True)
        return X[samples], y[samples]
    
    def fit(self, X, y, max_features=0):
        '''
        Обучает классификатор Случайный лес
        '''
        # Очистка списка деревьев при повторном запуске обучения
        if len(self.decision_trees) > 0:
            self.decision_trees = []
        self.max_features = max_features
        
        # Построение каждого дерева леса
        num_built = 0
        while num_built < self.num_trees:
            try:
                clf = DecisionTree(
                    min_samples_split=self.min_samples_split,
                    max_depth=self.max_depth
                )
                # Получаем подвыборку
                _X, _y = self._sample(X, y)
                # Обучаем
                clf.fit(_X, _y, self.max_features)
                # Сохраняем классификатор
                self.decision_trees.append(clf)
                num_built += 1
            except Exception as e:
                continue
        
    def predict(self, X):
        '''
        Предсказывает метку класса для новых объектов
        '''
        # Сделать предсказание каждым деревом в лесу
        y = []
        for tree in self.decision_trees:
            y.append(tree.predict(X))
            
        # Решейп для удобства поиска наиболее частого значения (строки, столбцы) == (объекты, решения деревьев)
        y = np.swapaxes(a=y, axis1=0, axis2=1)
        
        # Используем голосование большинством голосов для финального предсказания
        predictions = []
        for preds in y:
            counter = Counter(preds)
            predictions.append(counter.most_common(1)[0][0])
        return np.array(predictions)

In [113]:
X_test.shape

(30, 4)

In [118]:
model = RandomForest(num_trees=25, min_samples_split=2, max_depth=5)
model.fit(X_train, y_train, max_features=-1)
y_preds = model.predict(X_test)

In [119]:
 y_preds

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0])

In [116]:
y_test

array([1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2,
       0, 2, 2, 2, 2, 2, 0, 0])

In [117]:
accuracy_score(y_test, y_preds)

1.0